# RNN implementation in PYTHON

In [116]:
import numpy as np
import re

data = """
나라의 말이 중국과 달라 문자와 서로 통하지 아니하기에 이런 까닭으로 어리석은 백성이 이르고자 할 바가 있어도 마침내 제 뜻을 능히 펴지 못할 사람이 많으니라 내가 이를 위해 가엾이 여겨 새로 스물여덟 글자를 만드노니 사람마다 하여 쉬이 익혀 날로 씀에 편안케 하고자 할 따름이니라
"""


In [117]:
def data_preprocessing(data):
    data = re.sub('[^가-힣]', ' ', data)
    tokens = data.split()
    vocab = list(set(tokens))
    vocab_size = len(vocab)

    word_to_ix = {word: i for i, word in enumerate(vocab)}
    ix_to_word = {i: word for i, word in enumerate(vocab)}

    return tokens, vocab_size, word_to_ix, ix_to_word


In [118]:
def init_weights(h_size, vocab_size):
    U = np.random.randn(h_size, vocab_size) * 0.01
    W = np.random.randn(h_size, h_size) * 0.01
    V = np.random.randn(vocab_size, h_size) * 0.01
    return U,W,V

In [119]:
def feedforward(inputs, targets, hprev):
    loss = 0
    xs, hs, ps, ys = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    for i in range(seq_len):
        xs[i] = np.zeros((vocab_size, 1))
        xs[i][inputs[i]] = 1  # 각각의 word에 대한 one hot coding 
        hs[i] = np.tanh(np.dot(U, xs[i]) + np.dot(W, hs[i - 1]))
        ys[i] = np.dot(V, hs[i])
        ps[i] = np.exp(ys[i]) / np.sum(np.exp(ys[i]))  # softmax계산
        loss += -np.log(ps[i][targets[i], 0])
    return loss, ps, hs, xs

In [120]:
def backward(ps, hs, xs):

    # Backward propagation through time (BPTT)
    # 처음에 모든 가중치들은 0으로 설정
    dV = np.zeros(V.shape)
    dW = np.zeros(W.shape)
    dU = np.zeros(U.shape)

    for i in range(seq_len)[::-1]:
        output = np.zeros((vocab_size, 1))
        output[targets[i]] = 1
        ps[i] = ps[i] - output.reshape(-1, 1)
        # 매번 i스텝에서 dL/dVi를 구하기
        dV_step_i = ps[i] @ (hs[i]).T  # (y_hat - y) @ hs.T - for each step

        dV = dV + dV_step_i  # dL/dVi를 다 더하기

        # 각i별로 V와 W를 구하기 위해서는
        # 먼저 공통적으로 계산되는 부분을 delta로 해서 계산해두고
        # 그리고 시간을 거슬러 dL/dWij와 dL/dUij를 구한 뒤
        # 각각을 합하여 dL/dW와 dL/dU를 구하고 
        # 다시 공통적으로 계산되는 delta를 업데이트

        # i번째 스텝에서 공통적으로 사용될 delta
        delta_recent = (V.T @ ps[i]) * (1 - hs[i] ** 2)

        # 시간을 거슬러 올라가서 dL/dW와 dL/dU를 구하
        for j in range(i + 1)[::-1]:
            dW_ij = delta_recent @ hs[j - 1].T

            dW = dW + dW_ij

            dU_ij = delta_recent @ xs[j].reshape(1, -1)
            dU = dU + dU_ij

            # 그리고 다음번 j번째 타임에서 공통적으로 계산할 delta를 업데이트
            delta_recent = (W.T @ delta_recent) * (1 - hs[j - 1] ** 2)

        for d in [dU, dW, dV]:
            np.clip(d, -1, 1, out=d)
    return dU, dW, dV, hs[len(inputs) - 1]

In [121]:
def predict(word, length):
    x = np.zeros((vocab_size, 1))
    x[word_to_ix[word]] = 1
    ixes = []
    h = np.zeros((h_size,1))

    for t in range(length):
        h = np.tanh(np.dot(U, x) + np.dot(W, h))
        y = np.dot(V, h)
        p = np.exp(y) / np.sum(np.exp(y))    # 소프트맥스
        ix = np.argmax(p)                    # 가장 높은 확률의 index를 리턴
        x = np.zeros((vocab_size, 1))        # 다음번 input x를 준비
        x[ix] = 1
        ixes.append(ix)
    pred_words = ' '.join(ix_to_word[i] for i in ixes)
    return pred_words


In [122]:
# 기본적인 parameters
epochs = 10000
h_size = 100
seq_len = 3
learning_rate = 1e-2

In [123]:
tokens, vocab_size, word_to_ix, ix_to_word = data_preprocessing(data)

In [124]:
ix_to_word

{0: '이런',
 1: '쉬이',
 2: '아니하기에',
 3: '못할',
 4: '달라',
 5: '스물여덟',
 6: '만드노니',
 7: '편안케',
 8: '문자와',
 9: '펴지',
 10: '사람마다',
 11: '글자를',
 12: '까닭으로',
 13: '날로',
 14: '바가',
 15: '여겨',
 16: '백성이',
 17: '있어도',
 18: '사람이',
 19: '가엾이',
 20: '어리석은',
 21: '많으니라',
 22: '익혀',
 23: '새로',
 24: '말이',
 25: '내가',
 26: '씀에',
 27: '하여',
 28: '나라의',
 29: '이를',
 30: '통하지',
 31: '중국과',
 32: '제',
 33: '따름이니라',
 34: '서로',
 35: '능히',
 36: '하고자',
 37: '위해',
 38: '마침내',
 39: '할',
 40: '이르고자',
 41: '뜻을'}

In [126]:
U, W, V = init_weights(h_size, vocab_size)

In [127]:
p = 0
hprev = np.zeros((h_size, 1))
for epoch in range(epochs):

    for p in range(len(tokens)-seq_len):
        inputs = [word_to_ix[tok] for tok in tokens[p:p + seq_len]]
        targets = [word_to_ix[tok] for tok in tokens[p + 1:p + seq_len + 1]]

        loss, ps, hs, xs = feedforward(inputs, targets, hprev)

        dU, dW, dV, hprev = backward(ps, hs, xs)

        # Update weights and biases using gradient descent
        W -= learning_rate * dW
        U -= learning_rate * dU
        V -= learning_rate * dV

        # p += seq_len

    if epoch % 100 == 0:
        print(f'epoch {epoch}, loss: {loss}')

epoch 0, loss: 11.212981572514845
epoch 100, loss: 1.6656247872554224
epoch 200, loss: 0.26089094182579
epoch 300, loss: 0.12941435310994087
epoch 400, loss: 0.083728648341404
epoch 500, loss: 0.060332210674093945
epoch 600, loss: 0.04668973393112946
epoch 700, loss: 0.037888070483935
epoch 800, loss: 0.03171360352621403
epoch 900, loss: 0.027144998131597927
epoch 1000, loss: 0.02366621806051932
epoch 1100, loss: 0.020960934750283634
epoch 1200, loss: 0.018817025177428733
epoch 1300, loss: 0.017085778845490927
epoch 1400, loss: 0.015660027889379313
epoch 1500, loss: 0.014462273955098548
epoch 1600, loss: 0.013437359581322642
epoch 1700, loss: 0.012546637783123078
epoch 1800, loss: 0.011762911380583496
epoch 1900, loss: 0.011066571583932537
epoch 2000, loss: 0.010443114012686935
epoch 2100, loss: 0.009881617884749262
epoch 2200, loss: 0.009373675309357135
epoch 2300, loss: 0.008912515240743636
epoch 2400, loss: 0.008492298833980717
epoch 2500, loss: 0.008107651485150793
epoch 2600, loss

In [128]:
predict('나라의', 10)

'말이 중국과 달라 중국과 달라 문자와 달라 문자와 서로 통하지'